In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from dateutil.parser import parse

In [2]:
mpath = os.getcwd()
path = mpath+'/DAS output'
files = os.listdir(path)

In [3]:
cols = []
df = pd.DataFrame()
for f in files:
    if 'pivot' not in f:
        df1 = pd.read_csv(path+'/'+f)
        df = pd.concat([df,df1],axis=0)

In [50]:
df.to_csv('MetaPointsRaw.csv')

In [4]:
print(len(df))
df = df.drop_duplicates()
print(len(df))

8415314
8347020


In [5]:
dates = df['timestamp'].values
date = []
for d in dates:
    date.append(parse(d))
df['time']=date

In [6]:
df = df.query("name!='Module 07 - Status Codes - Max / Min Cell Voltage Location'")
df['core']=df['deviceName'].apply(lambda x: x[0:7])
df.loc[:,'first'] = df['name'].apply(lambda x: x.split('-')[0].strip())
df['second']=df['first'].apply(lambda x: x.split(' ')[0])

In [54]:
dfmod = df[df['second']=='Module']
dfmod.loc[:,'point']=dfmod['name'].apply(lambda x: x.split('-')[2].strip())
dfmod = dfmod.reset_index()
dfmod.drop(columns=['index'],inplace=True)
dfmod.reset_index(drop=True,inplace=True)

/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [55]:
df_mod = dfmod.pivot_table(index=['time','core','first'],columns='point',values='value').reset_index()

In [56]:
df_mod.loc[:,'module'] = df_mod['first'].apply(lambda x: int(x[-2:]))
df_mod.drop(columns='first',inplace=True)

In [60]:
cols = df_mod.columns.tolist()
col = cols[0:2]+cols[-2:]+cols[2:-2]
df_mod = df_mod[col]
df_mod.rename(columns={'time':'Time','core':'Core','module':'Module'},inplace=True)
df_mod

point,Time,Core,Rack Number,Module,Average Cell Temperature,Max Cell Temperature,Min Cell Temperature
0,2020-05-01 00:00:00-04:00,Core 03,7.0,1,18.5,18.5,18.5
1,2020-05-01 00:00:00-04:00,Core 03,7.0,2,19.5,19.5,19.5
2,2020-05-01 00:00:00-04:00,Core 03,7.0,3,20.0,20.0,20.0
3,2020-05-01 00:00:00-04:00,Core 03,7.0,4,21.0,21.0,21.0
4,2020-05-01 00:00:00-04:00,Core 03,7.0,5,21.5,22.0,21.5
...,...,...,...,...,...,...,...
2912498,2020-06-02 18:00:00-04:00,Core 04,15.0,13,27.0,27.0,27.0
2912499,2020-06-02 18:00:00-04:00,Core 04,15.0,14,NaN,27.0,27.0
2912500,2020-06-02 18:00:00-04:00,Core 04,15.0,15,27.0,27.0,27.0
2912501,2020-06-02 18:00:00-04:00,Core 04,15.0,16,26.7,27.0,26.5


In [66]:
dfcurr = df[df['second']=='Current']
dfcurr.loc[:,'sensor'] = dfcurr['first'].apply(lambda x: x.split(' ')[1]+'_'+x.split(' ')[3])
df_curr = dfcurr[['time','core','sensor','value']]
df_curr

/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,time,core,sensor,value
0,2020-05-02 12:00:00-04:00,Core 03,Humidity_1,35.8
1,2020-05-02 12:00:02-04:00,Core 03,Humidity_1,35.9
2,2020-05-02 12:00:06-04:00,Core 03,Humidity_1,35.8
3,2020-05-02 12:00:08-04:00,Core 03,Humidity_1,35.9
4,2020-05-02 12:00:10-04:00,Core 03,Humidity_1,35.8
...,...,...,...,...
15052,2020-05-03 11:55:34.092000-04:00,Core 03,Temperature_8,21.0
15053,2020-05-03 11:56:36.141000-04:00,Core 03,Temperature_8,21.0
15054,2020-05-03 11:57:36.141000-04:00,Core 03,Temperature_8,21.0
15055,2020-05-03 11:58:36.142000-04:00,Core 03,Temperature_8,21.0


In [84]:
dfhvac = df[df['second']=='HVAC']
dfhvac['HVAC']=dfhvac['first'].apply(lambda x: x.split(' ')[2])
dfhvac['point']=dfhvac['first'].apply(lambda x: x.split(' ')[3]+'_'+x.split(' ')[4])
df_hvac = dfhvac.pivot_table(index=['time','core','HVAC'],columns='point',values='value').reset_index()

/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/alejandrajolodosky/.asdf/installs/python/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [87]:
df_hvac.rename(columns={'time':'Time','core':'Core'},inplace=True)
df_hvac

point,Time,Core,HVAC,Cooling_1,Cooling_2,Current_Humidity,Current_Temperature
0,2020-05-01 00:00:00-04:00,Core 03,1,0.0,0.0,62.2,23.1
1,2020-05-01 00:00:00-04:00,Core 03,2,0.0,0.0,57.4,23.2
2,2020-05-01 00:00:01.863000-04:00,Core 03,1,NaN,0.0,NaN,NaN
3,2020-05-01 00:00:02-04:00,Core 03,1,NaN,NaN,62.1,NaN
4,2020-05-01 00:00:02-04:00,Core 03,2,NaN,NaN,57.3,NaN
...,...,...,...,...,...,...,...
150448,2020-05-03 23:59:56-04:00,Core 03,2,NaN,NaN,NaN,21.8
150449,2020-05-03 23:59:58-04:00,Core 03,1,NaN,NaN,NaN,21.6
150450,2020-05-03 23:59:58-04:00,Core 03,2,NaN,NaN,48.3,21.7
150451,2020-05-04 00:00:00-04:00,Core 03,1,0.0,0.0,49.2,21.6


In [88]:
df_mod.to_pickle('Module_Temps.pkl')
df_curr.to_pickle('Sensors.pkl')
df_hvac.to_pickle('HVAC.pkl')